# 01 Multidimensional multi Knapsack Problem

## Problem Overview

The multidimensional multi knapsack problem (MMKP) is a generalization of the classic knapsack problem. In this problem, we have multiple knapsacks, each with its own capacity constraints across multiple dimensions (e.g., weight, volume, etc.). We also have a set of items, each with a value and a set of weights corresponding to each dimension. The goal is to maximize the total value of the items placed in the knapsacks without exceeding the capacity constraints of any knapsack in any dimension. 

### Attributes
- The `CONSTRAINTS` array define for each knapsack, what is the maximum weight and what volume it can carry.
- The `WEIGHTS` array defines for each item, what are the dimensions of the item (weight, volume, ...).
- The `VALUES` array defines for each item, what is its value.

### Example Output

with 3 kapsacks and 10 items with 2 dimensions (weight and volume):

```bash
(array([[287, 117],
        [319, 114],
        [204, 244]]),
 array([[52, 69],
        [13, 86],
        [89, 76],
        [54, 52],
        [25, 89],
        [ 5, 87],
        [50, 19],
        [97, 60],
        [85, 27],
        [97, 79]]),
 array([64, 73, 52, 71,  0, 95,  0, 34, 31, 21]))
```

> Note: It is need less to say that the items are the indecises of the arrays like 0, 1, 2, ..., n-1 in `VALUES` and `WEIGHTS`. **and one items can be in only one knapsack.**

In [1]:
import numpy as np
from tqdm import tqdm
from collections import deque

In [17]:
# Problem 1 
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)
# inital solution with empty knapsacks
solution = np.array(
    [[False for _ in range(NUM_ITEMS)] for _ in range(NUM_KNAPSACKS)] , dtype=np.bool
)
# 
ACTION_SET = ['add','remove','swap']

In [18]:
def value(solution : np.ndarray) -> int:
    return (VALUES * np.any(solution, axis=0)).sum()

def valid(solution : np.ndarray) -> bool:
    return np.all(WEIGHTS.T@solution.T <= CONSTRAINTS.T)

def get_items_left(solution: np.ndarray):
    '''
    Since each item can only be in one knapsack, we just check indices of current solution for all knapsacks are empty, if so, that item is left outside of knapsacks.
    '''
    items_in_knapsacks = np.any(solution, axis=0)
    return np.where(~items_in_knapsacks)[0]

def get_items_in(solution: np.ndarray):
    '''
    Same idea as get_items_left function, but here we return indices of items that are in knapsacks.
    '''
    items_in_knapsacks = np.any(solution, axis=0)
    return np.where(items_in_knapsacks)[0]

In [ ]:
def action_tweak(solution: np.ndarray, action : str) -> np.ndarray:

    assert action in ACTION_SET, f"Action must be one of {ACTION_SET}"

    new_solution = solution.copy()

    if action == 'add':
        items_left = get_items_left(new_solution)
        if len(items_left) == 0:
            return new_solution, action
        item_to_add = rng.choice(items_left)
        knapsack_to_add = rng.integers(0, NUM_KNAPSACKS)
        new_solution[knapsack_to_add, item_to_add] = True
    elif action == 'remove':
        items_in = get_items_in(new_solution)
        if len(items_in) == 0:
            return new_solution, action
        item_to_remove = rng.choice(items_in)
        knapsack_to_remove = np.where(new_solution[:, item_to_remove])[0][0] # find which knapsack has this item
        new_solution[knapsack_to_remove, item_to_remove] = False
    elif action == 'swap':
        items_in = get_items_in(new_solution)
        if len(items_in) > 1:
            item_to_remove = rng.choice(items_in, size=2, replace=False)
            knapsack_to_remove_1 = np.where(new_solution[:, item_to_remove[0]])[0][0]
            knapsack_to_remove_2 = np.where(new_solution[:, item_to_remove[1]])[0][0]
            new_solution[knapsack_to_remove_1, item_to_remove[0]] = False
            new_solution[knapsack_to_remove_2, item_to_remove[1]] = False
            new_solution[knapsack_to_remove_1, item_to_remove[1]] = True
            new_solution[knapsack_to_remove_2, item_to_remove[0]] = True
        else:
            return new_solution, action

    
    return new_solution, action

In [28]:
def test_tweak(solution):
    new_solution = solution.copy()
    for _ in range(10):
        new_solution, act = action_tweak(new_solution, action=rng.choice(ACTION_SET, p=[0.6,0.1,0.3]))
        items_left = get_items_left(new_solution)
        print("Action:", act)
        print("Items left:", items_left, "Count:", len(items_left))
        print("Items in:", get_items_in(new_solution), "Count:", len(get_items_in(new_solution)))
        print("Value:", value(new_solution))
        print("Valid:", valid(new_solution))
        print(20 * "-")
        # print(new_solution)
    return new_solution
n_s = test_tweak(solution)

Action: swap
Items left: [0 1 2 3 4 5 6 7 8 9] Count: 10
Items in: [] Count: 0
Value: 0
Valid: True
--------------------
Action: swap
Items left: [0 1 2 3 4 5 6 7 8 9] Count: 10
Items in: [] Count: 0
Value: 0
Valid: True
--------------------
Action: add
Items left: [0 1 2 4 5 6 7 8 9] Count: 9
Items in: [3] Count: 1
Value: 43
Valid: False
--------------------
Action: add
Items left: [1 2 4 5 6 7 8 9] Count: 8
Items in: [0 3] Count: 2
Value: 51
Valid: False
--------------------
Action: add
Items left: [1 2 4 5 6 7 9] Count: 7
Items in: [0 3 8] Count: 3
Value: 71
Valid: False
--------------------
Action: add
Items left: [1 4 5 6 7 9] Count: 6
Items in: [0 2 3 8] Count: 4
Value: 136
Valid: False
--------------------
Action: add
Items left: [1 4 5 7 9] Count: 5
Items in: [0 2 3 6 8] Count: 5
Value: 144
Valid: False
--------------------
Action: add
Items left: [4 5 7 9] Count: 4
Items in: [0 1 2 3 6 8] Count: 6
Value: 221
Valid: False
--------------------
Action: add
Items left: [4 7 9] Cou

In [29]:
from collections import deque

def hill_climbing(solution: np.ndarray, max_iter=1000, max_messages=5, print_every=100):
    best_solution = solution.copy()
    best_value = value(best_solution)
    messages = deque(maxlen=max_messages)
    
    for iteration in tqdm(range(max_iter), desc="Hill Climbing"):
        new_solution, _ = action_tweak(best_solution, 
                                       action=rng.choice(ACTION_SET))

        items_left = len(get_items_left(new_solution))
        
        if valid(new_solution):
            new_value = value(new_solution)
            if new_value > best_value:
                best_value = new_value
                best_solution = new_solution
                messages.append(f"Iter {iteration}: value = {best_value}, item left = {items_left}")
            if new_value == best_value:
                best_solution = new_solution

        if iteration % print_every == 0 and messages:
            tqdm.write("\n".join(messages))
            messages.clear()

        if items_left == 0:
            tqdm.write("All items assigned!")
            break

    # tqdm.write(f"Iter {iteration + 1}: Best value = {best_value}, items left = {len(get_items_left(best_solution))}")
    return best_solution, best_value, items_left

# Probem 1 Configuration

In [35]:
# Problem 1 
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)
# inital solution with empty knapsacks
solution = np.array(
    [[False for _ in range(NUM_ITEMS)] for _ in range(NUM_KNAPSACKS)] , dtype=np.bool
)

best_solution, best_value, items_left_num = hill_climbing(solution, max_iter=100000)
print(100 * "-")
print("Best value found for problem 1:", best_value)
if items_left_num != 0:
    print("But there are still", items_left_num, "items left unassigned.")

Hill Climbing:   0%|          | 155/100000 [00:00<00:15, 6411.16it/s]

Iter 41: value = 591, item left = 9
Iter 42: value = 603, item left = 8
Iter 52: value = 611, item left = 7
Iter 55: value = 694, item left = 6
Iter 58: value = 772, item left = 5
All items assigned!
----------------------------------------------------------------------------------------------------
Best value found for problem 1: 1020


In [9]:
def sa_hill_climbing(solution: np.ndarray, max_iter=1000, max_messages=5,
                    print_every=100, initial_temp=1.0, cooling_rate=0.99):
    best_solution = solution.copy()
    best_value = value(best_solution)
    temp = initial_temp
    messages = deque(maxlen=max_messages)

    for iteration in tqdm(range(max_iter), desc="Simulated Annealing Hill Climbing"):
        new_solution = sa_tweak(best_solution, strength=0.8)
        items_left = len(get_items_left(new_solution))

        if valid(new_solution):
            new_value = value(new_solution)
            if new_value > best_value:
                best_value = new_value
                best_solution = new_solution
                messages.append(f"Iter {iteration}: value = {best_value}, item left = {items_left}")
            elif new_value == best_value: # moving is better than staying ! 
                best_solution = new_solution
            else: # SA 
                # accept with a probability of exp((new_value - best_value) / temp)
                prob = np.exp((new_value - best_value) / temp)
                if np.random.rand() < prob:
                    best_solution = new_solution

        temp *= cooling_rate

        if iteration % print_every == 0 and messages:
            tqdm.write("\n".join(messages))
            messages.clear()

        if items_left == 0:
            tqdm.write("All items assigned!")
            break

    return best_solution, best_value, items_left

# Problem 2 Configuration

In [42]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)
# inital solution with empty knapsacks
solution = np.array(
    [[False for _ in range(NUM_ITEMS)] for _ in range(NUM_KNAPSACKS)] , dtype=np.bool
)

best_solution, best_value, items_left_num = hill_climbing(solution, max_iter=1000000)
print(100 * "-")
print("Best value found for problem 2:", best_value)
if items_left_num != 0:
    print("But there are still", items_left_num, "items left unassigned.")

Hill Climbing:   0%|          | 2316/1000000 [00:00<01:25, 11689.34it/s]

Iter 92: value = 11857, item left = 75
Iter 93: value = 12493, item left = 74
Iter 94: value = 12990, item left = 73
Iter 95: value = 13428, item left = 72
Iter 100: value = 13495, item left = 71
Iter 174: value = 20838, item left = 57
Iter 175: value = 21665, item left = 56
Iter 177: value = 22469, item left = 55
Iter 183: value = 22944, item left = 54
Iter 189: value = 23783, item left = 53
Iter 260: value = 30962, item left = 40
Iter 261: value = 31740, item left = 39
Iter 264: value = 31966, item left = 38
Iter 265: value = 32296, item left = 37
Iter 275: value = 32762, item left = 36
Iter 356: value = 33620, item left = 35
Iter 376: value = 34337, item left = 34
Iter 415: value = 34707, item left = 33
Iter 469: value = 34846, item left = 32
Iter 575: value = 35209, item left = 31
Iter 952: value = 35840, item left = 30
Iter 976: value = 36302, item left = 29
Iter 1139: value = 36430, item left = 28
Iter 1154: value = 37211, item left = 27
Iter 1313: value = 37954, item left = 26
I

Hill Climbing:   1%|          | 5206/1000000 [00:00<01:13, 13588.92it/s]

Iter 2590: value = 42886, item left = 18


Hill Climbing:   1%|          | 8066/1000000 [00:00<01:10, 14035.61it/s]

...

Iter 189371: value = 47871, item left = 9


Hill Climbing:  43%|████▎     | 430820/1000000 [00:27<00:36, 15525.39it/s]

Iter 427983: value = 48554, item left = 8


Hill Climbing: 100%|██████████| 1000000/1000000 [01:04<00:00, 15598.52it/s]

----------------------------------------------------------------------------------------------------
Best value found for problem 2: 48554
But there are still 8 items left unassigned.


# Problem 3 

In [44]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)
# inital solution with empty knapsacks
solution = np.array(
    [[False for _ in range(NUM_ITEMS)] for _ in range(NUM_KNAPSACKS)] , dtype=np.bool
)

best_solution, best_value, items_left_num = hill_climbing(solution, max_iter=1000)
print(100 * "-")
print("Best value found for problem 3:", best_value)
if items_left_num != 0:
    print("But there are still", items_left_num, "items left unassigned.")

Hill Climbing:  10%|█         | 102/1000 [00:16<02:27,  6.09it/s]

Iter 86: value = 16372, item left = 4973
Iter 88: value = 17230, item left = 4972
Iter 91: value = 17573, item left = 4971
Iter 94: value = 18343, item left = 4970
Iter 95: value = 18932, item left = 4969


Hill Climbing:  20%|██        | 202/1000 [00:33<02:22,  5.59it/s]

Iter 173: value = 30230, item left = 4948
Iter 182: value = 31157, item left = 4947
Iter 186: value = 32069, item left = 4946
Iter 193: value = 32728, item left = 4945
Iter 194: value = 32734, item left = 4944


Hill Climbing:  30%|███       | 302/1000 [00:50<01:55,  6.06it/s]

...

Iter 790: value = 135083, item left = 4732
Iter 796: value = 135332, item left = 4731
Iter 798: value = 135400, item left = 4730
Iter 799: value = 135528, item left = 4729
Iter 800: value = 136018, item left = 4728


Hill Climbing:  90%|█████████ | 902/1000 [02:32<00:17,  5.76it/s]

Iter 880: value = 149250, item left = 4705
Iter 882: value = 149405, item left = 4704
Iter 886: value = 149454, item left = 4703
Iter 887: value = 149489, item left = 4702
Iter 898: value = 150198, item left = 4701


Hill Climbing: 100%|██████████| 1000/1000 [02:50<00:00,  5.88it/s]

----------------------------------------------------------------------------------------------------
Best value found for problem 3: 164741
But there are still 4671 items left unassigned.
